# Praca domowa nr 4

Dokument ten przedstawia implementację zapytań SQL z pracy domowej. Rozwiązania opierają się na pakietach numpy oraz pandas. Poprawność każdego z rozwiązań sprawdzam za pomocą funkcji 'equals'. Autor raportu: Piotr Fic

## Import pakietów oraz tymczasowa baza danych

In [1]:
import pandas as pd
import numpy as np

#Wczytanie ramek danych
posts = pd.read_csv("travel/Posts.csv.gz")
users = pd.read_csv("travel/Users.csv.gz")
votes = pd.read_csv("travel/Votes.csv.gz")
comments = pd.read_csv("travel/Comments.csv.gz")
badges = pd.read_csv("travel/Badges.csv.gz")

In [2]:
#Tymczasowa baza danych do zapytan SQL
import tempfile, sqlite3
import os, os.path
baza = os.path.join(tempfile.mkdtemp(), "travel.db")
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

posts.to_sql("Posts", conn)
users.to_sql("Users", conn)
votes.to_sql("Votes", conn)
comments.to_sql("Comments", conn)
badges.to_sql("Badges", conn)

# Zapytanie nr 1
Schemat działania:
 - Połączenie użytkowników z ich postami - wybieramy tylko pytania.
 - Grupujemy dane w zależności od użytkownika za pomocą "OwnerUserId"
 - Szukamy postów - pytań, które otrzymały najwięcej Likes oraz sumujemy "Likes" uzyskane we wszystkich pytaniach danego użytkownika 

Rezultat:
Znajdujemy użytkowników, którzy uzyskali w sumie najwięcej "Likes", wyświetlamy ich dane wraz z najwyżej ocenionym pytaniem.

In [3]:
#ZADANIE 1
#Zapytanie w SQL
sql1 = pd.read_sql_query("""
SELECT
    Users.DisplayName,
    Users.Age,
    Users.Location,
    SUM(Posts.FavoriteCount) AS FavoriteTotal,
    Posts.Title AS MostFavoriteQuestion,
    MAX(Posts.FavoriteCount) AS MostFavoriteQuestionLikes
FROM Posts
JOIN Users ON Users.Id=Posts.OwnerUserId
WHERE Posts.PostTypeId=1
GROUP BY OwnerUserId
ORDER BY FavoriteTotal DESC
LIMIT 10""", conn)

## Rozwiązanie

In [4]:
#Join Users oraz Posts po kolumnach Id == OwnerUserId
result = pd.merge(posts, users, left_on='OwnerUserId', right_on='Id')
#Wybor gdzie PostTypeId == 1
result = result.loc[result.PostTypeId==1]
#Wybor potrzebnych kolumn
result = result[['DisplayName', 'Age', 'Location', 'FavoriteCount', 'Title', 'OwnerUserId']]
#Suma oraz maksimum
tmp = result.groupby('OwnerUserId', sort=True).agg({'FavoriteCount': [np.max, np.sum]}).reset_index()
#Polaczenie sumy i maksow z ramka
tmp.columns = tmp.columns.droplevel()
tmp.columns = ['OwnerUserId', 'MostFavoriteQuestionLikes','FavoriteTotal']
result = pd.merge(result, tmp, left_on=['OwnerUserId', "FavoriteCount"], 
                  right_on=['OwnerUserId', 'MostFavoriteQuestionLikes'])
#Nazwanie kolumn, sortowanie
result = result.rename(columns={'Title': 'MostFavoriteQuestion'})
result = result.iloc[:, lambda result:[0,1,2,4,6,7]]
result.sort_values(by=['FavoriteTotal'], inplace=True, ascending=False)
result = result.iloc[:, lambda result:[0,1,2,5,3,4]]
result = result.head(10).reset_index(drop=True)

#Sprawdzenie poprawnosci
result.equals(sql1)

True

# Zapytanie nr 2
Schemat działania:
 - Z odpowiedzi wybieramy ocenione pozytywnie (Score > 0), zliczamy liczbę pozytywnych odpowiedzi dla danego pytania
 - Łączymy zliczone pozytywne odpowiedzi z pytaniami za pomocą ParentId
 
Rezultat:
Znajdujemy pytania o największej liczbie pozytywnie ocenionych odpowiedzi.

In [5]:
#ZADANIE 2
#Zapytanie SQL
sql2 = pd.read_sql_query("""
SELECT
    Posts.ID,
    Posts.Title,
    Posts2.PositiveAnswerCount
FROM Posts
JOIN (
    SELECT
        Posts.ParentID,
    COUNT(*) AS PositiveAnswerCount
    FROM Posts
    WHERE Posts.PostTypeID=2 AND Posts.Score>0
    GROUP BY Posts.ParentID
    ) AS Posts2
ON Posts.ID=Posts2.ParentID
ORDER BY Posts2.PositiveAnswerCount DESC
LIMIT 10""", conn)

## Rozwiązanie

In [9]:
#Pomocnicza ramka Posts2
Posts2 = posts.loc[(posts.PostTypeId==2) & (posts.Score>0)]
#Zliczenie pogrupowanych postow
Posts2 = Posts2.groupby('ParentId').size().reset_index()
#Nazwy kolumn
Posts2.columns = ['ParentId', 'PositiveAnswerCount']
#Wybor w ramce Posts
result = posts[['Id', 'Title']]
#Polaczenie posts i posts2
result = pd.merge(posts, Posts2, left_on = ['Id'], right_on = ['ParentId'])
result = result[['Id','Title','PositiveAnswerCount']]
result.sort_values(by=['PositiveAnswerCount','Id'], inplace=True, ascending=[False, True])
result = result.head(10).reset_index(drop=True)

#Sprawdzenie poprawnosci
result.equals(sql2)

True

# Zapytanie nr 3
Schemat działania:
 - Zliczamy roczne UpVotes dla postów
 - Łączymy zliczone UpVotes z postami, wybieramy pytania.
 - W zależności od roku wybieramy pytanie o największej liczbie UpVotes

Rezultat:
Znajdujemy pytania które w danym roku otrzymały najwięcej UpVotes.


In [10]:
#ZADANIE 3
#Zapytanie SQL
sql3 = pd.read_sql_query("""
SELECT
    Posts.Title,
    UpVotesPerYear.Year,
    MAX(UpVotesPerYear.Count) AS Count
FROM (
    SELECT
        PostId,
        COUNT(*) AS Count,
        STRFTIME('%Y', Votes.CreationDate) AS Year
    FROM Votes
    WHERE VoteTypeId=2
    GROUP BY PostId, Year
    ) AS UpVotesPerYear
JOIN Posts ON Posts.Id=UpVotesPerYear.PostId
WHERE Posts.PostTypeId=1
GROUP BY Year""", conn)

## Rozwiązanie

In [12]:
#Ramka UpVotesPerYear
UpVotesPerYear = votes.loc[votes.VoteTypeId==2]
#Zmiana daty na rok
UpVotesPerYear.loc[:, 'CreationDate'] = UpVotesPerYear.CreationDate.str.slice(0,4)
#Zliczanie po grupowaiu
UpVotesPerYear = UpVotesPerYear.groupby(['PostId','CreationDate']).size().reset_index()
#Wybor kolumn
UpVotesPerYear.columns = ['PostId', 'Year', 'Count']
#Ramka pomocnicza
tmp = posts.loc[posts.PostTypeId==1]
tmp = tmp[['Title','Id']]
#Polaczenie do wynikowej ramki
result = pd.merge(tmp, UpVotesPerYear, left_on=['Id'], right_on=['PostId'])
#Policzenie max i wybor potrzebnych kolumn
tmp2 = result.groupby('Year').agg({'Count': np.max}).reset_index()
result = pd.merge(result, tmp2, on = ['Year', 'Count'])
result = result[['Title', 'Year', 'Count']]

#Sprawdzenie poprawnosci
result.equals(sql3)

True

# Zapytanie nr 4
Schemat działania:
 - Wybieramy odpowiedzi o maksymalnej ocenie (MaxScore) grupując po zadanym pytaniu (ParentId)
 - Łączymy powyższe odpowiedzi z pytaniami na które odpowiadały
 - Mamy zbiór pytań z informacją o najwyżej ocenionej odpowiedzi, łączymy go z postami które zostały oznaczone jako AcceptedAnswer dla danego pytania
 - Obliczamy różnicę pomiędzy najlepszą oceną ze wszystkich odpowiedzi na dane pytanie (MaxScore), a oceną odpowiedzi oznaczonej jako AcceptedAnswer na dane pytanie

Rezultat:
Znajdujemy pytania gdzie wystąpiła największa różnica pomiędzy najwyżej oceniona odpowiedzią, a odpowiedzią oznaczoną jako AcceptedAnswer.

In [13]:
#ZADANIE 4
#Zapytanie SQL
sql4 = pd.read_sql_query("""
SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT * FROM Posts
WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC""", conn)

## Rozwiązanie

In [14]:
#Ramka BestAnswers
BestAnswers = posts.loc[posts.PostTypeId==2]
#Liczymy max w kolumnie Score
tmp = BestAnswers.groupby('ParentId').agg({'Score': np.max}).reset_index()
#Laczymy z poczatkowa ramka
BestAnswers = pd.merge(BestAnswers, tmp, on=['ParentId','Score'])
#Wybor i nazwanie kolumn
BestAnswers = BestAnswers.iloc[:, lambda result:[7,12,14]]
BestAnswers = BestAnswers.rename(columns={'Score': 'MaxScore'})

#Ramka Questions i merge z BestAnswers
Questions = posts.loc[posts.PostTypeId==1]
tmp = pd.merge(BestAnswers, Questions, left_on='ParentId', right_on='Id')
#Ramka wynikowa
result = pd.merge(posts, tmp, left_on='Id', right_on='AcceptedAnswerId')
#Wybor kolumn po mergu
result = result[['Id_y', 'Title_y', 'MaxScore', 'Score_x']]
#Utworzenie kolumn Difference
result["Difference"]=result['MaxScore']-result['Score_x']
#Sortowanie po Difference
result.sort_values(by = ['Difference'], inplace=True, ascending=False)
result = result.reset_index(drop=True)
result.columns=['Id','Title','MaxScore','AcceptedScore','Difference']
result = result.loc[result.Difference>50]

#Sprawdzenie poprawnosci
result.equals(sql4)

True

# Zapytanie nr 5
Schemat działania:
 - Obliczamy sumaryczną ocenę komentarzy dla danego posta w zależności od komentującego
 - Łączymy powyższe dane z danymi wszystkich postów tak że:
    - komentarze odpowiadają postowi
    - komentującym jest "właściciel posta"
    - wybieramy tylko pytania

Rezultat:
Otrzymujemy pytania do których komentarze dodane przez samego pytającego miały sumarycznie najwyższą ocenę.

In [15]:
#ZADANIE 5
#Zapytanie SQL
sql5 = pd.read_sql_query("""
SELECT
Posts.Title,
CmtTotScr.CommentsTotalScore
FROM (
SELECT
PostID,
UserID,
SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostID, UserID
) AS CmtTotScr
JOIN Posts ON Posts.ID=CmtTotScr.PostID AND Posts.OwnerUserId=CmtTotScr.UserID
WHERE Posts.PostTypeId=1
ORDER BY CmtTotScr.CommentsTotalScore DESC
LIMIT 10""", conn)

## Rozwiązanie

In [16]:
#Ramka CmtTotScr
CmtTotScr = comments.groupby(['PostId', 'UserId']).agg({'Score': np.sum})
CmtTotScr = CmtTotScr.rename(columns={'Score': 'CommentsTotalScore'})
#Polaczenie do ramki wynikowej
result = pd.merge(posts, CmtTotScr, left_on=['Id','OwnerUserId'], right_on=['PostId','UserId'])
result.sort_values(by = ['CommentsTotalScore','Id'],ascending=[False,True], inplace=True)
#Wybor PostTypeId
result = result.loc[result.PostTypeId==1]
#Wybor kolumn i sortowanie
result = result[['Title', 'CommentsTotalScore']]
#result.sort_values(by = ['CommentsTotalScore'], inplace=True, ascending=False)
result = result.head(10).reset_index(drop=True)

#Sprawdzenie poprawnosci
result.equals(sql5)

True

# Zapytanie nr 6
Schemat działania:
 - Z informacji na temat przyznawanych odznak wybieramy użytkowników, którym przyznano odznakę klasy 1 od 2 do 10 razy
 - Łącząc z informacjami z Users, wybieramy kolumny danych oraz wiersze unikatowe, aby nie powtarzać User-ów

Rezultat:
Informacje o użytkownikach, którym odznakę klasy 1 przyznano od 2 do 10 razy.

In [17]:
#ZADANIE 6
#Zapytanie SQL
sql6 = pd.read_sql_query("""
SELECT DISTINCT
    Users.Id,
    Users.DisplayName,
    Users.Reputation,
    Users.Age,
    Users.Location
FROM (
    SELECT
        Name, UserID
    FROM Badges
    WHERE Name IN (
        SELECT
            Name
        FROM Badges
        WHERE Class=1
        GROUP BY Name
        HAVING COUNT(*) BETWEEN 2 AND 10
    )
    AND Class=1
    ) AS ValuableBadges
JOIN Users ON ValuableBadges.UserId=Users.Id""", conn)

## Rozwiązanie

In [20]:
#Pomocnicza ramka do zliczenia
tmp = badges.loc[badges.Class==1]
#Zliczanie po Name
tmp = tmp.groupby('Name').size().reset_index()
tmp = tmp.rename(columns={0:"Count"})
#Wybor pomiedzy 2 a 10 dla Count
tmp = tmp.loc[(tmp.Count>=2) & (tmp.Count<=10)]
#Wartosci z Name do listy
tmp = tmp["Name"].tolist()
#Ramka ValuableBadges
ValuableBadges = badges[badges.Name.isin(tmp)]
ValuableBadges = ValuableBadges.loc[ValuableBadges.Class==1]
#Wybor kolumn
ValuableBadges = ValuableBadges[['Name', 'UserId']]
#Ramka wynikowa
result = pd.merge(ValuableBadges, users, left_on='UserId', right_on='Id')
#Wybor kolumn oraz wartosci unikatowych
result = result.iloc[:,lambda tmp:[7,5,10,3,9]].drop_duplicates().reset_index(drop=True)

#Sprawdzenie poprawnosci
result.equals(sql6)

True

# Zapytanie nr 7
Schemat działania:
 - Przyjmujemy głosy z lat 2016 - 2017 za ”nowe”, wcześniejsze za ”stare”, wybieramy tylko UpVotes oraz zliczamy w zależności od postu i roku
 - Grupując dla danego postu obliczamy:
    - liczbę UpVotes z lat 2016-17
    - liczbę UpVotes z lat przed 2016
    - sumaryczną liczbę UpVotes
 - Łącząc zebrane dane z postami wybieramy pytania, które nie uzyskały UpVotes w latach 2016-17

Rezultat:
Otrzymujemy pytania, które przed rokiem 2016 uzyskały najwięcej UpVotes, a w latach 2016-17 nie otrzymały żadnego.

In [21]:
#ZADANIE 7
#Zapytanie SQL
sql7 = pd.read_sql_query("""
SELECT
    Posts.Title,
    VotesByAge2.OldVotes
FROM Posts
JOIN (
    SELECT
        PostId,
        MAX(CASE WHEN VoteDate = 'new' THEN Total ELSE 0 END) NewVotes,
        MAX(CASE WHEN VoteDate = 'old' THEN Total ELSE 0 END) OldVotes,
        SUM(Total) AS Votes
    FROM (
        SELECT
            PostId,
            CASE STRFTIME('%Y', CreationDate)
                WHEN '2017' THEN 'new'
                WHEN '2016' THEN 'new'
                ELSE 'old'
                END VoteDate,
            COUNT(*) AS Total
        FROM Votes
        WHERE VoteTypeId=2
        GROUP BY PostId, VoteDate
    ) AS VotesByAge
    GROUP BY VotesByAge.PostId
    HAVING NewVotes=0
) AS VotesByAge2 ON VotesByAge2.PostId=Posts.ID
WHERE Posts.PostTypeId=1
ORDER BY VotesByAge2.OldVotes DESC
LIMIT 10""", conn)

## Rozwiązanie

In [24]:
#Ramka VotesByAge
VotesByAge = votes.loc[votes.VoteTypeId==2]
#Zamiana daty na rok
VotesByAge.loc[:, 'CreationDate'] = VotesByAge.CreationDate.str.slice(0,4)
#Przypisanie 'new' 'old'
VotesByAge.loc[:, 'CreationDate'] = np.where((VotesByAge['CreationDate'] == '2016') | (VotesByAge['CreationDate'] == '2017'), 'new', 'old')
#Zmiana nazwy kolumny
VotesByAge = VotesByAge.rename(columns={'CreationDate': 'VoteDate'}).reset_index(drop=True)
#Zliczenie jako total
VotesByAge = VotesByAge.groupby(['PostId','VoteDate']).size().reset_index()
VotesByAge.columns = ['PostId','VoteDate','Total']
#Kolumny NewVotes i OldVotes
VotesByAge["NewVotes"] = np.where(VotesByAge['VoteDate']=='new', VotesByAge['Total'], 0)
VotesByAge["OldVotes"] = np.where(VotesByAge['VoteDate']=='old', VotesByAge['Total'], 0)
#Maksy i suma
VotesByAge2 = VotesByAge.groupby(['PostId']).agg({'Total': np.sum, 'NewVotes': np.max, 'OldVotes': np.max}).reset_index()
VotesByAge2 = VotesByAge2.loc[VotesByAge2.NewVotes==0]
#Ramka wynikowa
result = pd.merge(VotesByAge2, posts, left_on = 'PostId', right_on = 'Id')
#SOrtujemy dla zachowania kolejnosci jak SQL
result.sort_values(by = ['OldVotes','PostId'], ascending=[False,True], inplace=True)
result = result.loc[result.PostTypeId==1]
#Wybor kolumn
result = result[['Title', 'OldVotes']]
#Sortowanie wedlug OldVotes
result.sort_values(by = ['OldVotes'], inplace=True, ascending=False)
result = result.head(10).reset_index(drop=True)

#Sprawdzenie poprawnosci
result.equals(sql7)

True